In [1]:
# import importlib
# import db_utils
# importlib.reload(db_utils)

In [2]:
%pip install langchain-elasticsearch langchain-community langchain_experimental langchain_openai tqdm pypdf streamlit

Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain_community.embeddings import FastEmbedEmbeddings
from langchain_community.llms.ollama import Ollama

from db_utils import add_documents_to_db, remove_index, remove_document, fetch_all
from files_utils import chunk_documents
from model_utils import rag_prompt
from config import MODEL_NAME, ES_PORT, ES_INDEX_NAME, ES_DISTANCE_STRATEGY, CONTEXT_CHAR_THRESHOLD, CHUNKER_TYPE
import pickle

Prepare chunks

In [5]:
pdf_directory = "./data/college"

document_chunks = chunk_documents(pdf_directory=pdf_directory, chunker_type=CHUNKER_TYPE)

Loaded 96 documents.


In [6]:
document_chunks

[Document(page_content='Akademia Górniczo-Hutnicza w Krakowie\nWstając z kolan upadliśmy na ryj.\nO społeczności AGH\nW wypadkach śmiertelnych uczestniczyłem już wiele razy.\nProwadzący o sesji\nAkademia Górniczo-Hutnicza imienia Stanisława\nStaszica w  Piotrkowie Trybunalskim  Krakowie  –\nuczelnia raczej wyższa  położona w południowej Polsce.\nPołożona na zachód od centrum, południe od', metadata={'source': 'data\\college\\Akademia Górniczo-Hutnicza w Krakowie – Nonsensopedia, polska encyklopedia humoru.pdf', 'page': 0, 'id': 'data\\college\\Akademia Górniczo-Hutnicza w Krakowie – Nonsensopedia, polska encyklopedia humoru.pdf:0:0'}),
 Document(page_content='Akademia Górniczo-Hutnicza imienia Stanisława\nStaszica w  Piotrkowie Trybunalskim  Krakowie  –\nuczelnia raczej wyższa  położona w południowej Polsce.\nPołożona na zachód od centrum, południe od\npółnocnego zachodu i odśrodkowo od hipocentrum,\nktórego jest ona epicentrum . Założona w 1919  roku jako\ndobudówka do MS AGH, stąd na

In [7]:
len(document_chunks)

852

In [8]:
# save document_chunks to a file
with open("document_chunks_FastEmbedEmbeddings_college.pkl", "wb") as f:
    pickle.dump(document_chunks, f)

In [2]:
with open("document_chunks_college.pkl", "rb") as f:
    loaded_document_chunks = pickle.load(f)
    
loaded_document_chunks

[Document(page_content='Akademia Górniczo-Hutnicza w Krakowie\nWstając z kolan upadliśmy na ryj. O społeczności AGH\nW wypadkach śmiertelnych uczestniczyłem już wiele razy. Prowadzący o sesji\nAkademia Górniczo-Hutnicza imienia Stanisława\nStaszica w  Piotrkowie Trybunalskim  Krakowie  –\nuczelnia raczej wyższa  położona w południowej Polsce. Położona na zachód od centrum, południe od\npółnocnego zachodu i odśrodkowo od hipocentrum,\nktórego jest ona epicentrum .', metadata={'source': 'data\\college\\Akademia Górniczo-Hutnicza w Krakowie – Nonsensopedia, polska encyklopedia humoru.pdf', 'page': 0, 'id': 'data\\college\\Akademia Górniczo-Hutnicza w Krakowie – Nonsensopedia, polska encyklopedia humoru.pdf:0:0'}),
 Document(page_content='Założona w 1919  roku jako\ndobudówka do MS AGH, stąd nazwa. Wydziały\nZamknąć to se możecie ryje. WIMiR to ostatni\nbastion. Starosta po rozmowie z dziekanem o\nkoronawirusie\nWydział Elektr otechniki, Automatyki, Informatyki  (wdech)  i Inżynierii Biome

In [3]:
document_chunks = loaded_document_chunks

In [ ]:
#document_chunks = document_chunks[50:75] # limit chunks

In [8]:
document_chunks

[Document(page_content='Akademia Górniczo-Hutnicza w Krakowie\nWstając z kolan upadliśmy na ryj.\nO społeczności AGH\nW wypadkach śmiertelnych uczestniczyłem już wiele razy.\nProwadzący o sesji\nAkademia Górniczo-Hutnicza imienia Stanisława\nStaszica w  Piotrkowie Trybunalskim  Krakowie  –\nuczelnia raczej wyższa  położona w południowej Polsce.\nPołożona na zachód od centrum, południe od', metadata={'source': 'data\\college\\Akademia Górniczo-Hutnicza w Krakowie – Nonsensopedia, polska encyklopedia humoru.pdf', 'page': 0, 'id': 'data\\college\\Akademia Górniczo-Hutnicza w Krakowie – Nonsensopedia, polska encyklopedia humoru.pdf:0:0'}),
 Document(page_content='Akademia Górniczo-Hutnicza imienia Stanisława\nStaszica w  Piotrkowie Trybunalskim  Krakowie  –\nuczelnia raczej wyższa  położona w południowej Polsce.\nPołożona na zachód od centrum, południe od\npółnocnego zachodu i odśrodkowo od hipocentrum,\nktórego jest ona epicentrum . Założona w 1919  roku jako\ndobudówka do MS AGH, stąd na

Prepare database

In [9]:
embedding = FastEmbedEmbeddings()

db_kwargs = {
    "embedding": embedding,
    "es_url": "http://localhost:9200",
    "index_name": "rag",
    "distance_strategy": "COSINE"
}

c:\Users\Dodek\Desktop\AGH\WDZB\projekt\RAG-System\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 5 files: 100%|██████████| 5/5 [00:00<?, ?it/s]


In [5]:
# clear index 
remove_index(index_name=db_kwargs["index_name"], db_config={"hosts": db_kwargs["es_url"]})

Index 'rag' deleted successfully.


In [10]:
db = add_documents_to_db(db=None, document_chunks=document_chunks, db_kwargs=db_kwargs, bulk_upload=True)

Bulk ingesting documents...
Done


In [11]:
fetch_all(
    index_name=db_kwargs["index_name"],
    db_config={"hosts": db_kwargs["es_url"]},
)

Retrieved 1077 documents.
{'text': 'Akademia Górniczo-Hutnicza w Krakowie\nWstając z kolan upadliśmy na ryj. O społeczności AGH\nW wypadkach śmiertelnych uczestniczyłem już wiele razy. Prowadzący o sesji\nAkademia Górniczo-Hutnicza imienia Stanisława\nStaszica w  Piotrkowie Trybunalskim  Krakowie  –\nuczelnia raczej wyższa  położona w południowej Polsce. Położona na zachód od centrum, południe od\npółnocnego zachodu i odśrodkowo od hipocentrum,\nktórego jest ona epicentrum . Założona w 1919  roku jako\ndobudówka do MS AGH, stąd nazwa. Wydziały\nZamknąć to se możecie ryje.', 'metadata': {'source': 'data\\college\\Akademia Górniczo-Hutnicza w Krakowie – Nonsensopedia, polska encyklopedia humoru.pdf', 'page': 0, 'id': 'data\\college\\Akademia Górniczo-Hutnicza w Krakowie – Nonsensopedia, polska encyklopedia humoru.pdf:0:0'}}
{'text': 'WIMiR to ostatni\nbastion. Starosta po rozmowie z dziekanem o\nkoronawirusie\nWydział Elektr otechniki, Automatyki, Informatyki  (wdech)  i Inżynierii Biome

Prompt rag

In [11]:
model = Ollama(model=MODEL_NAME)

response = rag_prompt(
    query = "Jaki wydział jest potocznie nazywany CPN -em?",
    model=model,
    db=db,
    chunker_type=CHUNKER_TYPE,
)

# remove_document(
#     index_name=db_kwargs["index_name"],
#     source_file="",
#     db_config={"hosts": db_kwargs["es_url"]},
#     )


Kontekst: W metanolu, bo etanolu szkoda.

---

• Wydział Geodezji Górniczej i Inżynierii Środowiska – wszyscy mylą słowa składowe z 
WGGiOŚ. • Wydział Geologii, Geofizyki i Ochrony Środowiska – wszyscy mylą słowa składowe z 
WGGiIŚ. Jego siedzibą jest budynek A -0, czyli najbardziej prestiżowy a zarazem oddalony 
od reszty kampusu budynek. • Wydział Inżynierii Lądowej i Gospodarki Zasobami – wydział pierwszy i najstarszy, do 
2021 Wydział Górnictwa i Geoinżynierii. Nazwa zmieniona na nazwę bez sensu, ale grunt 
że nie ma w nazwie górnictwa, które jest obecnie passé. • Wydział Humanistyczny – też nas to dziwi. • Wydział Informatyki, Elektroniki i Telekomunikacji – nikt nic o nim nie wie. • Wydział Inżynierii Materiałowej i Ceramiki – stąd pochodzi Jego Miłościwość Rektor oraz 
zastawa twojej babci. • Wydział Inżynierii Mechanicznej i Robotyki – obiekt prześmiewek przez studentów 
zarówno innych wydziałów jak i swoich własnych. Ostatni bastion podczas pandemii 
(zajęcia były tu prowadzon